In [201]:
# initialisation
import pandas as pd
import numpy as np
import plotly.express as px
import itertools
from scipy.stats import *
from pandasgui import show
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [202]:
# load the data
raw_data = pd.read_csv('../data/Games Database - Main Page.csv')
#show(raw_data)
#drop fields we won't be needing
raw_data.drop(['Me - Review', 'Percentile', 'Launch Platform(s)', 'Hours Played', 'Completion', 'Year(s) revisited/ continued/ earlier dabbled'], axis = 1, inplace = True)
raw_data

,Game,Rating,Reviewscore,Played on,Launch Year,Play Year,DLC/ Major update played,Publisher,Developer,Franchise,Key words,Remaster Reviewscore,Remaster Release Year,Developer2
0,The Legend of Zelda: Tears of the Kingdom,100,96,Switch,2023,2023,No,Nintendo,Nintendo,The Legend of Zelda,"Adventure, open-world, sandbox, exploration, p...",NaN,NaN,NaN
1,The Legend of Zelda: Breath of the Wild,100,97,Switch,2017,2017,Yes,Nintendo,Nintendo,The Legend of Zelda,"Adventure, open-world, sandbox, exploration, p...",NaN,NaN,NaN
2,God of War (2018),99,94,PS4,2018,2018,No,Sony,Santa Monica Studio,God of War,"Hack and slash, third-person combat, story-driven",NaN,NaN,NaN
3,The Last of Us,99,95,PS4,2013,2014,Yes,Sony,Naughty Dog,The Last of Us,"Horror, TPS, stealth, survival, story-driven, ...",94.0,2014.0,NaN
4,Metroid Prime,99,97,Switch,2002,2023,No,Nintendo,Retro Studios,Metroid,"Metroidvania, FPS, platforming, exploration",94.0,2023.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,Fifa 06 Game Boy Advance,44,49,GBA,2005,2005,No,EA,EA sports,Fifa,Sports (football),NaN,NaN,NaN
434,Cruis'n Blast,42,73,Switch,2021,2021,No,Indie,Raw Thrills,Cruise'n Blast,Racing (arcade),NaN,NaN,NaN
435,Fifa 14 Legacy Edition,38,38,Vita,2013,2013,No,EA,EA Sports,Fifa,Sports (football),NaN,NaN,NaN
436,Sunlight,34,78,PC,2021,2023,No,Indie,Krillbite Studio,Sunlight,Adventure,NaN,NaN,NaN


In [203]:
# a bit of basic plotting
fig = px.scatter(raw_data, x="Reviewscore", y="Rating", color="Played on", hover_name="Game")
fig

In [204]:
#create set of unique key words from the data frame
str_data = raw_data.copy()
str_data['Key words'] = str_data['Key words'].apply(lambda x: x.split(',')) #convert each entry in key words to a list
lst = str_data['Key words'].to_list() #append all lists together
combine = sum(lst, []) #roll the lists into one
remove_space = [s.strip(' ') for s in combine] #remove leading and trailing spaces in list items
capitalize = [s.upper() for s in remove_space] #convert all characters to upper case
kwords = set(capitalize) #remove duplicate list items
kwords = list(kwords)

# add new columns which track the keywords
kw_data = raw_data.copy()
kw_data['Key words'] = kw_data['Key words'].str.upper()
for i in kwords:
    kw_data[i] = kw_data['Key words'].apply(lambda x: 1 if str(i) in x else 0)

In [205]:
#create set of unique platforms from the data frame
str_data = raw_data.copy()
str_data['Played on'] = str_data['Played on'].apply(lambda x: x.split(',')) #convert each entry in key words to a list
lst = str_data['Played on'].to_list() #append all lists together
combine = sum(lst, []) #roll the lists into one
remove_space = [s.strip(' ') for s in combine] #remove leading and trailing spaces in list items
capitalize_plats = [s.upper() for s in remove_space] #convert all characters to upper case
plats = set(capitalize_plats) #remove duplicate list items
plats = list(plats)

# add new columns which track the keywords
data = kw_data.copy()
data['Played on'] = data['Played on'].str.upper()
for i in plats:
    data[i] = data['Played on'].apply(lambda x: 1 if str(i) in x else 0)

In [206]:
# calculate play - launch field
data['Play - launch'] = data['Play Year'] - data['Launch Year']
data['Play - launch'].fillna(0, inplace=True)
# calculate upgraded played field
data.loc[np.isnan(data['Remaster Reviewscore']) == False, 'Remaster played'] = 1
data['Remaster played'].fillna(0, inplace=True)
# calculate any multiplayer field
data['Key words'] = data['Key words'].str.upper()
data.loc[data['Key words'].str.contains('MULTIPLAYER') == True, 'Any multiplayer'] = 1
data['Any multiplayer'].fillna(0, inplace=True)
# calculate any racing field
data.loc[data['Key words'].str.contains('RACING') == True, 'Any racing'] = 1
data['Any racing'].fillna(0, inplace=True)
# calculate any sports field
data.loc[data['Key words'].str.contains('SPORTS') == True, 'Any sports'] = 1
data['Any sports'].fillna(0, inplace=True)
# calculate any simulation field
data.loc[data['Key words'].str.contains('SIMULATION') == True, 'Any simulation'] = 1
data['Any simulation'].fillna(0, inplace=True)
# calculate any strategy field
data.loc[data['Key words'].str.contains('STRATEGY') == True, 'Any strategy'] = 1
data['Any strategy'].fillna(0, inplace=True)


In [207]:
# convert the yes no fields to 1 or 0
data['DLC/ Major update played'] = data['DLC/ Major update played'].replace({'Yes': 1, 'No': 0})

In [208]:
#replace publisher string with its average review score
publisher_avg = (data[['Publisher','Reviewscore']]).groupby(['Publisher']).mean()['Reviewscore']
data = pd.merge(data, publisher_avg, on = 'Publisher')
data.rename(columns= {'Reviewscore_y' : 'Publisher avg', 'Reviewscore_x' : 'Reviewscore'}, inplace = True)

#replace developer string with its average review score
developer_avg = (data[['Developer','Reviewscore']]).groupby(['Developer']).mean()['Reviewscore']
data = pd.merge(data, developer_avg, on = 'Developer')
data.rename(columns= {'Reviewscore_y' : 'Developer avg', 'Reviewscore_x' : 'Reviewscore'}, inplace = True)

#replace franchise string with its average review score
franchise_avg = (data[['Franchise','Reviewscore']]).groupby(['Franchise']).mean()['Reviewscore']
data = pd.merge(data, franchise_avg, on = 'Franchise')
data.rename(columns= {'Reviewscore_y' : 'Franchise avg', 'Reviewscore_x' : 'Reviewscore'}, inplace = True)


In [209]:
# tidy missing values
data['Remaster Reviewscore'].fillna(0, inplace=True)

In [210]:
#have another look at the data
data

,Game,Rating,Reviewscore,Played on,Launch Year,Play Year,DLC/ Major update played,Publisher,Developer,Franchise,...,Play - launch,Remaster played,Any multiplayer,Any racing,Any sports,Any simulation,Any strategy,Publisher avg,Developer avg,Franchise avg
0,The Legend of Zelda: Tears of the Kingdom,100,96,SWITCH,2023,2023,0,Nintendo,Nintendo,The Legend of Zelda,...,0,0.0,0.0,0.0,0.0,0.0,0.0,85.555556,87.25,92.0
1,The Legend of Zelda: Breath of the Wild,100,97,SWITCH,2017,2017,1,Nintendo,Nintendo,The Legend of Zelda,...,0,0.0,0.0,0.0,0.0,0.0,0.0,85.555556,87.25,92.0
2,The Legend of Zelda: Ocarina of Time,91,99,3DS,1998,2023,0,Nintendo,Nintendo,The Legend of Zelda,...,25,1.0,0.0,0.0,0.0,0.0,0.0,85.555556,87.25,92.0
3,The Legend of Zelda: Skyward Sword,84,93,SWITCH,2011,2021,0,Nintendo,Nintendo,The Legend of Zelda,...,10,1.0,0.0,0.0,0.0,0.0,0.0,85.555556,87.25,92.0
4,The Legend of Zelda: Link's Awakening,73,87,SWITCH,2019,2019,0,Nintendo,Nintendo,The Legend of Zelda,...,0,0.0,0.0,0.0,0.0,0.0,0.0,85.555556,87.25,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,Human Fall Flat,62,68,PC,2016,2020,0,Curve Digital,Tomas Sakalauskas,Human Fall Flat,...,4,0.0,1.0,0.0,0.0,0.0,0.0,70.750000,68.00,68.0
434,Spacelines from the Far Out,60,82,PC,2022,2022,0,Skystone Games,Coffeenauts,Spacelines from the Far Out,...,0,0.0,1.0,0.0,0.0,0.0,0.0,82.000000,82.00,82.0
435,Finding Nemo,56,63,GBA,2003,2004,0,THQ,Vicarious Visions,Finding Nemo,...,1,0.0,0.0,0.0,0.0,0.0,0.0,63.000000,63.00,63.0
436,Lost Ark,54,82,PC,2022,2022,0,Amazon Games,Tripod Studio,Lost Ark,...,0,0.0,0.0,0.0,0.0,0.0,0.0,82.000000,82.00,82.0


In [211]:
# prep for model
X = data.copy()
X.drop(['Rating', 'Game', 'Played on', 'Publisher', 'Developer', 'Franchise', 'Key words', 'Remaster Release Year', 'Developer2'], axis = 1, inplace = True)
y = data['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [212]:
#initialize and fit model
model = GradientBoostingRegressor(random_state=0)
model.fit(X_train, y_train)

GradientBoostingRegressor(random_state=0)

In [213]:
# obtain feature importances
importance = model.feature_importances_
importances_df = pd.DataFrame([importance], columns = X.columns)
importances_df = importances_df.transpose()
importances_df.to_csv('../data/feature_importances.csv') # send importances to csv
importances_df

,0
Reviewscore,0.525510
Launch Year,0.023524
Play Year,0.053388
DLC/ Major update played,0.032375
Remaster Reviewscore,0.004781
...,...
Any simulation,0.000000
Any strategy,0.000188
Publisher avg,0.020853
Developer avg,0.094577


In [214]:
# analyse performance using test set
test_labels = pd.merge(y_test, data['Game'], left_index= True, right_index = True) #join the game name back in to rating scores
test_scores = pd.DataFrame({'Game': test_labels['Game'], 'Predicted Rating':model.predict(X_test), 'Actual Rating': test_labels['Rating']}) #join the test set to the predictions
test_scores.to_csv('../data/test_scores_output.csv')

In [215]:
# scatter plot of scores vs actual
fig = px.scatter(test_scores, x="Actual Rating", y="Predicted Rating", hover_name= 'Game')
fig

In [216]:
#output some actual model performance scores
print('Coefficient of determination - ', model.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model.predict(X_test)))

Coefficient of determination -  0.6260285759990798
Pearson R -  PearsonRResult(statistic=0.7948663506780815, pvalue=3.5431071391519047e-25)


In [217]:
#obtain plot of predicted rating vs review score
all_labels = pd.merge(y, data['Game'], left_index= True, right_index = True) #join the game name back in to rating scores
scores_labels = pd.merge(all_labels, data['Reviewscore'], left_index= True, right_index = True) #join the review avg back in to rating scores
scores_comp = pd.DataFrame({'Game': scores_labels['Game'], 
                            'Predicted Rating': model.predict(X), 
                            'Actual Rating': scores_labels['Rating'], 
                            'Review avg': scores_labels['Reviewscore']}) #join the test set to the predictions
# scatter plot
fig = px.scatter(scores_comp, x="Review avg", y="Predicted Rating", hover_name= 'Game')
fig
